<a href="https://colab.research.google.com/github/ipediz/BachehayekhafaneTMU/blob/main/Data_Adjustment_Casual_ready.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Removing the Counties where Lending Club was not available form the Dataset


In [9]:
!pip install graphviz
!pip install causalinference
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from matplotlib import style
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
from causalinference import CausalModel

import graphviz as gr

%matplotlib inline

style.use("fivethirtyeight")
pd.set_option("display.max_columns", 6)

In [10]:

Data = pd.read_csv("The_new_all_state_replication_data.csv")
Data['interaction'] = Data['interaction'] / 100
Data['percent_over_60'] = Data['percent_over_60'] / 100
# setting 'county_code' as index and 'new_timeperiod' as time



In [14]:
# Filter out rows where lending_club_available is 0
Data1_filtered_1= Data[Data['lending_club_available'] != 0]

# Verify the change
print(Data1_filtered_1.head())
print("Number of rows after filtering:", Data.shape[0])


   county_code  new_timeperiod  lending_club_available  ...  \
4         1019               5                       1  ...   
5         1019               6                       1  ...   
6         1019               7                       1  ...   
7         1019               8                       1  ...   
8         1019               9                       1  ...   

   business_bankruptcy_percapita  percent_over_60  interaction  
4                       0.000000         0.248477     0.248477  
5                       0.000000         0.248477     0.248477  
6                       2.857143         0.248477     0.248477  
7                       0.000000         0.248477     0.248477  
8                       0.000000         0.265622     0.265622  

[5 rows x 33 columns]
Number of rows after filtering: 64652


In [15]:
Data1_filtered_1.to_csv("Data1_filtered1.csv", index=False)

print("Filtered data has been saved to Data1_filtered.csv")

Filtered data has been saved to Data1_filtered.csv


The reason is, I want to see the effect of % of eldery population on the banckruptcies per capita, when Lending club is available. So, From the rest of the proccess, I will remove the counties where lending club is not available.

In [16]:
# Calculate mean of 'percent_over_60' for each 'new_timeperiod'
mean_percent_over_60_by_timeperiod = Data1_filtered_1.groupby('new_timeperiod')['percent_over_60'].mean()

# Display the result
print(mean_percent_over_60_by_timeperiod)


new_timeperiod
5     0.213747
6     0.213747
7     0.213747
8     0.213747
9     0.228495
10    0.228495
11    0.228495
12    0.228495
13    0.244676
14    0.244676
15    0.244676
16    0.244676
17    0.261559
18    0.261559
19    0.261559
20    0.261559
21    0.279346
22    0.279242
23    0.279242
24    0.279242
25    0.298510
26    0.298510
27    0.297551
28    0.297551
Name: percent_over_60, dtype: float64


I have the mean of each % over 60 in each county where lendingclub is available. Now, I want to have a binary variable in each county based on low or high % of population over 60.

In [18]:
# Calculate the mean of 'percent_over_60' for each 'new_timeperiod'
mean_percent_over_60_by_timeperiod = Data1_filtered_1.groupby('new_timeperiod')['percent_over_60'].mean()

# Define a function to assign 0 or 1 based on the condition
def assign_low_high(row):
    # Check if the value is less than the mean for the given new_timeperiod
    if row['percent_over_60'] < mean_percent_over_60_by_timeperiod[row['new_timeperiod']]:
        return 0
    else:
        return 1

# Apply the function to each row and create the new column 'Low-High_%_over60'
Data1_filtered_1['Low-High_%_over60'] = Data1_filtered_1.apply(assign_low_high, axis=1)

# Display the updated DataFrame to verify the changes
print(Data1_filtered_1[['county_code', 'new_timeperiod', 'percent_over_60', 'Low-High_%_over60']].head())


   county_code  new_timeperiod  percent_over_60  Low-High_%_over60
4         1019               5         0.248477                  1
5         1019               6         0.248477                  1
6         1019               7         0.248477                  1
7         1019               8         0.248477                  1
8         1019               9         0.265622                  1


In [19]:
# Save the updated DataFrame to a CSV file
Data1_filtered_1.to_csv("All_STATE_CASUAL_READY.csv", index=False)

print("The updated data has been saved to 'All_STATE_CASUAL_READY.csv'.")


The updated data has been saved to 'All_STATE_CASUAL_READY.csv'.


Now we have a file, where lendingclub is available, and the %of population over 60 is devided into a binary variable, low and high. (0 and 1)